In [1]:
import requests
import json
import time
import os
import pandas as pd
import dlt
from dotenv import load_dotenv

In [2]:
# === CONFIGURATION ===
load_dotenv()
CLIENT_ID = os.getenv(f'CLIENT_ID')
CLIENT_SECRET = os.getenv(f'CLIENT_SECRET')

TOKENS_FILE = 'secrets/strava_tokens.json'

In [3]:
# === LOAD OR REFRESH TOKENS ===
def load_tokens():
    if os.path.exists(TOKENS_FILE):
        with open(TOKENS_FILE, 'r') as f:
            return json.load(f)
    else:
        raise FileNotFoundError("Token file not found. Authorize first and save your tokens.")

def save_tokens(tokens):
    with open(TOKENS_FILE, 'w') as f:
        json.dump(tokens, f)

def refresh_tokens(tokens):
    if time.time() > tokens['expires_at']:
        print("Access token expired. Refreshing...")
        response = requests.post("https://www.strava.com/oauth/token", data={
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
            'grant_type': 'refresh_token',
            'refresh_token': tokens['refresh_token']
        })
        new_tokens = response.json()
        tokens.update({
            'access_token': new_tokens['access_token'],
            'refresh_token': new_tokens['refresh_token'],
            'expires_at': new_tokens['expires_at']
        })
        save_tokens(tokens)
    return tokens

# === GET ACTIVITIES ===
def get_activities(access_token, per_page=30):
    headers = {'Authorization': f"Bearer {access_token}"}
    page = 1

    while True:
        response = requests.get(
            'https://www.strava.com/api/v3/athlete/activities',
            headers=headers,
            params={'per_page': per_page, 'page': page}
        )
        data = response.json()

        if not data:
            break

        for activity in data:
            yield activity
        
        page += 1

In [4]:
# === MAIN FLOW ===
tokens = load_tokens()
tokens = refresh_tokens(tokens)
ACCESS_TOKEN=tokens['access_token']

activities_generator = get_activities(ACCESS_TOKEN)

activities = [activity for activity in activities_generator]

# === DISPLAY RESULTS ===
for act in activities:
    print(f"{act['start_date'][:10]} - {act['name']} - {act['distance']/1000:.2f} km")

2025-05-02 - Morning Run - 6.43 km
2025-05-02 - Morning Walk - 1.33 km
2025-04-30 - Morning Run - 4.53 km
2025-04-30 - Morning Walk - 1.30 km
2025-04-29 - Morning Run - 6.44 km
2025-04-29 - Morning Walk - 1.31 km
2025-04-28 - Morning Walk - 1.32 km
2025-04-27 - Morning Run - 6.44 km
2025-04-26 - Evening Walk - 1.72 km
2025-04-25 - Morning Run - 6.46 km
2025-04-25 - Morning Walk - 1.30 km
2025-04-23 - Evening Run - 1.77 km
2025-04-23 - Morning Walk - 1.33 km
2025-04-22 - Morning Run - 6.44 km
2025-04-20 - Lunch Run - 9.65 km
2025-04-19 - Morning Run - 6.11 km
2025-04-18 - Morning Walk - 1.32 km
2025-04-16 - Morning Run - 4.97 km
2025-04-15 - Morning Run - 6.44 km
2025-04-15 - Morning Walk - 1.33 km
2025-04-14 - Morning Walk - 1.21 km
2025-04-13 - Lunch Run - 9.25 km
2025-04-12 - Morning Run - 10.50 km
2025-04-04 - Afternoon Run - 10.19 km
2025-04-02 - Morning Run - 6.52 km
2025-04-01 - Morning Run - 10.38 km
2025-03-31 - Morning Run - 9.88 km
2025-03-28 - Morning Run - 10.79 km
2025-03-

In [5]:
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\william.heidel\\AppData\\Roaming',
        'APPLICATIONINSIGHTS_CONFIGURATION_CONTENT': '{}',
        'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': '1',
        'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey',
        'CHOCOLATEYLASTPATHUPDATE': '133586205342552078',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_23700_TSVXPBQNOLKTIWJJ',
        'CLIENT_ID': '157579',
        'CLIENT_SECRET': 'cbe88340d741962bc1e2a56e64b10e643c00dd83',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPONENT_INSTALLER_DIR': 'C:\\Users\\william.heidel\\AppData\\Local\\Microsoft\\Edge\\User Data',
        'COMPUTERNAME': 'CR-XLT798YPN3',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'CONDA_PREFIX': 'C:\\Users\\william.heidel\\.conda\

In [7]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import OffsetPaginator

os.environ["ACCESS_TOKEN"] = ACCESS_TOKEN
@dlt.resource(name="activities")
def paginated_getter(
    access_token=dlt.secrets.value,
):
    client = RESTClient(
        base_url='https://www.strava.com/api/v3/',
        auth=BearerTokenAuth(token=access_token),
        paginator=OffsetPaginator(
            limit=100,
            limit_param='per_page',
            offset=1,
            offset_param='page',
            stop_after_empty_page=True,
            total_path=None
        )
    )

    for page in client.paginate("athlete/activities"):
        for activity in page:
            yield activity

# Run the pull
for activity in paginated_getter():
    print(f"{activity['start_date'][:10]} - {activity['name']} - {activity['distance']/1000:.2f} km")


2025-05-02 - Morning Run - 6.43 km
2025-05-02 - Morning Walk - 1.33 km
2025-04-30 - Morning Run - 4.53 km
2025-04-30 - Morning Walk - 1.30 km
2025-04-29 - Morning Run - 6.44 km
2025-04-29 - Morning Walk - 1.31 km
2025-04-28 - Morning Walk - 1.32 km
2025-04-27 - Morning Run - 6.44 km
2025-04-26 - Evening Walk - 1.72 km
2025-04-25 - Morning Run - 6.46 km
2025-04-25 - Morning Walk - 1.30 km
2025-04-23 - Evening Run - 1.77 km
2025-04-23 - Morning Walk - 1.33 km
2025-04-22 - Morning Run - 6.44 km
2025-04-20 - Lunch Run - 9.65 km
2025-04-19 - Morning Run - 6.11 km
2025-04-18 - Morning Walk - 1.32 km
2025-04-16 - Morning Run - 4.97 km
2025-04-15 - Morning Run - 6.44 km
2025-04-15 - Morning Walk - 1.33 km
2025-04-14 - Morning Walk - 1.21 km
2025-04-13 - Lunch Run - 9.25 km
2025-04-12 - Morning Run - 10.50 km
2025-04-04 - Afternoon Run - 10.19 km
2025-04-02 - Morning Run - 6.52 km
2025-04-01 - Morning Run - 10.38 km
2025-03-31 - Morning Run - 9.88 km
2025-03-28 - Morning Run - 10.79 km
2025-03-